In [1]:
import torch
from torch.utils.data import DataLoader
import lava.lib.dl.slayer as slayer
from nmnist import NMNIST, SNN, augment

         Only Prophesee DVS demo will not run properly.
         Please install it from https://github.com/prophesee-ai/prophesee-automotive-dataset-toolbox


In [2]:
trained_dir = './Train'
nmnist = NMNIST(trained_dir, 3, 300, transform=augment, download=True)
nmnist_test = NMNIST(trained_dir, 3, 300)

device = torch.device('cuda')

net = SNN().to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

nmnist_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

nmnist_test_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

error = slayer.loss.SpikeRate(
    true_rate=0.2, false_rate=0.03, reduction='sum'
).to(device)

stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(
    net, error, optimizer, stats,
    classifier=slayer.classifier.Rate.predict, count_log=True
)

epochs = 1

for epoch in range(epochs):
    for i, (input, label) in enumerate(nmnist_loader):
        output, count = assistant.train(input, label)
        header = [
            'TRAIN\nEvent rate : ' +
            ', '.join([f'{c.item():.4f}' for c in count.flatten()]),
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)

    for i, (input, label) in enumerate(nmnist_test_loader):
        output, count = assistant.test(input, label)
        header = [
            'TEST\nEvent rate : ' +
            ', '.join([f'{c.item():.4f}' for c in count.flatten()]),
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)

    if stats.testing.best_accuracy:
        torch.save(net.state_dict(), trained_dir + '/network.pt')

    stats.update()

net.export_hdf5('network.net')

Attempting download...



     0K .......... .......... .......... .......... ..........  0%  292K 56m25s
    50K .......... .......... .......... .......... ..........  0%  547K 43m15s
   100K .......... .......... .......... .......... ..........  0%  892K 34m59s
   150K .......... .......... .......... .......... ..........  0%  924K 30m42s
   200K .......... .......... .......... .......... ..........  0% 2.70M 25m45s
   250K .......... .......... .......... .......... ..........  0% 2.79M 22m25s
   300K .......... .......... .......... .......... ..........  0% 1.16M 21m12s
   350K .......... .......... .......... .......... ..........  0% 2.01M 19m33s
   400K .......... .......... .......... .......... ..........  0% 4.06M 17m49s
   450K .......... .......... .......... .......... ..........  0% 2.03M 16m49s
   500K .......... .......... .......... .......... ..........  0% 4.28M 15m38s
   550K .......... .......... .......... .......... ..........  0% 2.67M 14m50s
   600K .......... .......... .........

Extracting files ...


FileNotFoundError: [Errno 2] No such file or directory: './Train.zip'

In [3]:
nmnist.example_of_each_class(assistant)